# The Unitary Events Analysis

The executed version of this tutorial is at https://elephant.readthedocs.io/en/latest/tutorials/unitary_event_analysis.html

The Unitary Events (UE) analysis \[1\] tool allows us to  reliably detect correlated spiking activity that is not explained by the firing rates of the neurons alone. It was designed to detect coordinated spiking activity that occurs significantly more often than predicted by the firing rates of the neurons. The method allows one to analyze correlations not only between pairs of neurons but also between multiple neurons, by considering the various spike patterns across the neurons. In addition, the method allows one to extract the dynamics of correlation between the neurons by perform-ing the analysis in a time-resolved manner. This enables us to relate the occurrence of spike synchrony to behavior.

The algorithm:

1. Align trials, decide on width of analysis window.
2. Decide on allowed coincidence width.
3. Perform a sliding window analysis. In each window:
    1. Detect and count coincidences.
    2. Calculate expected number of coincidences.
    3. Evaluate significance of detected coincidences.
    4. If significant, the window contains Unitary Events.
4. Explore behavioral relevance of UE epochs.

References:

1. Grün, S., Diesmann, M., Grammont, F., Riehle, A., & Aertsen, A. (1999). Detecting unitary events without discretization of time. Journal of neuroscience methods, 94(1), 67-79.

In [ ]:
import random
import string
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import neo

import elephant.unitary_event_analysis as ue
from elephant.datasets import download_datasets
from viziphant.unitary_event_analysis import plot_ue

# Fix random seed to guarantee fixed output
random.seed(1224)

%matplotlib inline

Next, we download a data file containing spike train data from multiple trials of two neurons.

In [ ]:
# Download data
repo_path='tutorials/tutorial_unitary_event_analysis/data/dataset-1.nix'
filepath=download_datasets(repo_path)

# Load data and extract spiketrains

In [ ]:
io = neo.io.NixIO(f"{filepath}",'ro')
block = io.read_block()

spiketrains = []
# each segment contains a single trial
for ind in range(len(block.segments)):
    spiketrains.append (block.segments[ind].spiketrains)


# Calculate Unitary Events

In [ ]:
UE = ue.jointJ_window_analysis(
    spiketrains, bin_size=5*pq.ms, win_size=100*pq.ms, win_step=10*pq.ms, pattern_hash=[3])

plot_ue(spiketrains, UE, significance_level=0.05)
plt.show()